<a href="https://colab.research.google.com/github/zhappo/Analise-de-dados-CRMV/blob/main/2023_crmvba_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 - Instruções de uso


---

##1.1 Obtenção e preparação dos arquivso a serem carregados
Os arquivos devem ser obtidos do SISCAD WEB em formato de excel. Após baixar os mesmos, abra retire a primeira linha em branco e salve eles em formato padrão do excel (xlsx)

##1.2 Execução do programa
Vá no menu superior em "Ambiente de execução" selecione executar tudo para executar o programa (ou aperte CtRL+F9) e carregue os arquivos preparados conforme forem sendo solicitados.

Serão gerados na seção 2.1 gráficos demonstrando a relação de:
   
* Estabelecimentos classificados  como Pessoa Jurídica (PJ) demonstrativo de 
atuanets e não atuantes (baixados)
* Estabelecimentos classificados  como cadastro (CD) demonstrativo de atuanets e não atuantes (baixados)
*Um gráico do tipo explosão solar mostrando a relação de estabelecimentos por atividades, tanto para os atuantes quanto para os não atuantes.

Será gerado na seção 2.2 um tabela demonstrando a relação de estabelecimentso das atividades predefinidas no municipio onde estão instalados.



---
##Observações
A tabela da seção 2.2 exibe os municiios conforme se encontram no cadastro




#2 - Execução do programa

In [28]:
#@title Carrega as bibliotecas necessarias para o programa funcionar
#Bibliotecas para importar arquivo
from google.colab import files
import io

#para analisar dados
import pandas as pd

#para gerar graficos
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [29]:
#@title Funções e Classes


def nomeDoArquivo(arquivo_carregado):
  nomeDoArquivo = list(arquivo_carregado.keys())
  nomeDoArquivo = nomeDoArquivo[0]
  return nomeDoArquivo

def criaDF(nomeDoArquivo):
  arquivoDF = pd.read_excel(f'/content/{nomeDoArquivo}')
  return arquivoDF

class Relatorio:
  def __init__ (self, arquivo_carregado):
    self.nome_do_arquivo = nomeDoArquivo(arquivo_carregado)
    self.df = criaDF(self.nome_do_arquivo).drop_duplicates(subset='Registro')
    self.Total = self.df.shape[0]
    self.Atuantes = self.df.loc[self.df['Atuante'] == "Sim"].shape[0]
    self.Baixadas = self.df.loc[self.df['Atuante'] != "Sim"].shape[0]

  #Diferença
  def diferenca(self,comparar_com):
    diferenca = self.df[~self.df.Registro.isin(comparar_com.Registro)]
    return diferenca
  
  #Interecessão
  def intercessao(self,comparar_com):
    intercessao = self.df[self.df.Registro.isin(comparar_com.Registro)]
    return intercessao


def atuantes(dataframe_base):
  atuantes = dataframe_base.loc[dataframe_base['Atuante']=='Sim'].shape[0]
  return atuantes

def baixadas(dataframe_base):
  atuantes = dataframe_base.loc[dataframe_base['Atuante']!='Sim'].shape[0]
  return atuantes

#diferença entre dataframes

def diferenca(dataframe1, dataframe2):
  diferenca = dataframe1[~dataframe1.Registro.isin(dataframe2.Registro)]
  return diferenca

def intercessao(dataframe1, dataframe2):
  intercessao = dataframe1[dataframe1.Registro.isin(dataframe2.Registro)]
  return intercessao

## Carrega os relatórios

In [17]:
#@title Carrega o arquivo de relatório geral 
geral_file = files.upload()

Saving cruz-geral.xlsx to cruz-geral.xlsx


In [18]:
#@title Carrega o arquivo de relatório de comércio
comercio_file = files.upload()

Saving cruz comercio.xlsx to cruz comercio.xlsx


In [19]:
#@title Carrega o arquivo de relatório de Prática
pratica_file = files.upload()

Saving cruz pratica.xlsx to cruz pratica.xlsx


In [20]:
#@title Carrega o arquivo de relatório de Agroindústria
agroindustria_file = files.upload()

Saving cruz agro.xlsx to cruz agro.xlsx


##Tratamento do dados

In [21]:
#Cria os Objetos
geral = Relatorio(geral_file)
comercio = Relatorio(comercio_file)
pratica = Relatorio(pratica_file)
agroindustria = Relatorio(agroindustria_file)

In [22]:
#Cria dataframe de classes
PJ = geral.df.loc[geral.df["Classe"] == "PJ"]
CD = geral.df.loc[geral.df["Classe"] != "PJ"]

##2.1 Visualização em Gráficos

In [23]:
#@title Estabelcimentos por situação
#



#Graficos Sunburst

fig_por_situacao = make_subplots(rows=1, cols=2, specs=[[{'type':'sunburst'}, {'type':'sunburst'}]])

#Sunbusrt 1x1
fig_por_situacao.add_trace(go.Sunburst(
    labels=['Pessoa Jurídica', 'Atuantes','Não Atuantes' ],
    parents=['','Pessoa Jurídica','Pessoa Jurídica'],
    values=[PJ.shape[0],atuantes(PJ) ,baixadas(PJ)]),
              1, 1)
#Sunburst 1x2
fig_por_situacao.add_trace(go.Sunburst(
    labels=['Cadastros', 'Atuantes','Não Atuantes' ],
    parents=['','Cadastros','Cadastros'],
    values=[CD.shape[0],atuantes(CD),baixadas(CD)]),
              1, 2)

In [24]:
#@title Visão sobre Atuantes

'''
Um gráfico do tipo explosão solar (sunburst) demonstrando quantitativos por ramos de atividade
'''

outros_atuantes = atuantes(PJ) - (agroindustria.Atuantes + comercio.Atuantes + atuantes(pratica.diferenca(comercio.df)))

#@title Visão Geral

fig =go.Figure(go.Sunburst(
    labels=[ "Estabelecimentos",
            "Pessoas Jurídica", "Cadastro",
            "Agroindústria","Comércio", "Prática sem Comércio","Outros",
            "Sem Atividades Veterinárias", "Com Atividades veterinárias",
                          
            ],
    parents=["",
             "Estabelecimentos", "Estabelecimentos",
            "Pessoas Jurídica","Pessoas Jurídica","Pessoas Jurídica","Pessoas Jurídica",
             "Comércio","Comércio",
             
             ],
    values=[geral.Atuantes,
            atuantes(PJ), atuantes(CD),
            agroindustria.Atuantes,comercio.Atuantes, atuantes(pratica.diferenca(comercio.df)),outros_atuantes,
            atuantes(comercio.diferenca(pratica.df)),atuantes(comercio.intercessao(pratica.df)),
            
           
            ],
    branchvalues="total",
))

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0),
                  height=600,
                  )

fig.show()

In [25]:
#@title Visão sobre Não Atuantes

outros_baixadas = baixadas(PJ) - (agroindustria.Baixadas + comercio.Baixadas + baixadas(pratica.diferenca(comercio.df)))

#@title Visão Geral

fig =go.Figure(go.Sunburst(
    labels=[ "Estabelecimentos",
            "Pessoas Jurídica", "Cadastro",
            "Agroindústria","Comércio", "Prática sem Comércio","Outros",
            "Sem Atividades Veterinárias", "Com Atividades veterinárias",
                          
            ],
    parents=["",
             "Estabelecimentos", "Estabelecimentos",
            "Pessoas Jurídica","Pessoas Jurídica","Pessoas Jurídica","Pessoas Jurídica",
             "Comércio","Comércio",
             
             ],
    values=[geral.Baixadas,
            baixadas(PJ), baixadas(CD),
            agroindustria.Baixadas,comercio.Baixadas, baixadas(pratica.diferenca(comercio.df)),outros_baixadas,
            baixadas(comercio.diferenca(pratica.df)),baixadas(comercio.intercessao(pratica.df)),
            
           
            ],
    branchvalues="total",
))

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0),
                  height=600,
                  )

fig.show()

##2.2 Visão em forma de tabela

Nesta seção será criada um tabela mostrando as distribuição de estabelecimentos conforme atividades desempenhadas, agrupados por municipios onde estão localizados.

Partindo da lista de municípios que será exibida é possivél também identificar nomes divergentes de um mesmo municiio cadastrado no sistema, o qual poderá auxiliar na correção para emissão de melhores relatórios em futuras execuções

In [26]:
#@title Lista com as cidades

lista_de_cidades = list(geral.df['Cidade'].drop_duplicates())

#Cria a lista que vai receber os dados por cidade
dados_tabela = [] 

#variaveis temporarias
nr_comercio = nr_outros = nr_total = 0


for cidade in lista_de_cidades:
  nr_agroindustria = atuantes(agroindustria.df.loc[agroindustria.df['Cidade'] == cidade])
  nr_pratica = atuantes(pratica.df.loc[pratica.df['Cidade'] == cidade])
  nr_comercio = atuantes(comercio.diferenca(pratica.df).loc[comercio.diferenca(pratica.df)['Cidade']== cidade])
  nr_total = atuantes(geral.df.loc[geral.df['Cidade'] == cidade])
  nr_outros = nr_total - (nr_agroindustria + nr_pratica + nr_comercio)
  
  dados_tabela.append(
      [
      cidade, #município
      nr_agroindustria, #Agroindustria
      nr_pratica, # Prática
      nr_comercio, # Comércio
      nr_outros, #Outros
      nr_total, # Total de Estabelecimentos  no município
      ]
  )

#Lista dos rotulos da tabela
rotulos_tabela = ['Cidade','Agroindústria','Atividades Veterinárias', 'Comércio', 'Outros', 'Total']

#Cria a tabela

tabela = pd.DataFrame(dados_tabela, columns=rotulos_tabela)

In [27]:
#@title Tabela 
tabela

,Cidade,Agroindústria,Atividades Veterinárias,Comércio,Outros,Total
0,CRUZ DAS ALMAS,2,8,13,3,26
